Softmax Classification

In [195]:
import torch

In [196]:
torch.manual_seed(1)

Softmax

In [197]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = torch.nn.functional.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [198]:
print(hypothesis.sum())

tensor(1.)


In [199]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = torch.nn.functional.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [200]:
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


In [201]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #dim 1에 대해 y를 unsqueeze 해서 1을 scatter 함

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [202]:
#cost 는 여러 방식으로 표현 가능
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

print(torch.log(torch.nn.functional.softmax(z, dim=1)))
# 를 torch.nn.functional.log_softmax를 사용해서 아래와 같이 표현 가능
print(torch.nn.functional.log_softmax(z, dim = 1))

# 따라서 아래와도 같이 표현 가능 (NLL = Negative Log Likelihood)

cost = torch.nn.functional.nll_loss(torch.nn.functional.softmax(z, dim=1), y)

# 또, torch.nn.functional.cross_entropy로도 표현 가능

cost = torch.nn.functional.cross_entropy(z, y)

# cost 를 이렇게 계산할 경우에는 loss에 softmax 연산이 들어가기 때문에
# Neural Network 에서 softmax가 빠져 있어야 하고, 
# 그렇게 되면 나중에 확률을 구할 때 softmax를 어자피 넣어주어야 할 상황이 나옴
# 또, 이런 상황에서는 one hot vector를 만드는 과정을 생략함

print(cost)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)
tensor(1.4689, grad_fn=<NllLossBackward0>)


In [203]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)



In [204]:
W = torch.zeros([4, 3], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = torch.optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

In [205]:
for epoch in range(nb_epochs + 1):
    hypothesis = torch.nn.functional.softmax(x_train.matmul(W) + b, dim=1)

    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean() # 강의와 다른데 이게 맞는 것 같음
    # 또는
    z = x_train.matmul(W) + b
    cost = torch.nn.functional.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


High-level Implementation with torch.nn.Module

In [206]:
class SoftmaxClassifierModel(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(4, 3)
    
    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

In [207]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)

    cost = torch.nn.functional.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358734
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
